# Example 4: Comparing Reconstruction Methods

This example compares different deconvolution methods: Wiener, Lucy-Richardson, and Tikhonov.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from frame_overlap import Workflow

%matplotlib inline

## Wiener Filter

In [ ]:
wf_wiener = Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)

(wf_wiener
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 25])
    .reconstruct(kind='wiener', noise_power=0.01))

stats_wiener = wf_wiener.recon.get_statistics()
print("Wiener Filter:")
print(f"  χ²/dof: {stats_wiener['chi2_per_dof']:.2f}")
print(f"  RMSE: {stats_wiener['rmse']:.2e}")
print(f"  MAE: {stats_wiener['mae']:.2e}")

## Lucy-Richardson Algorithm

In [ ]:
wf_lucy = Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)

(wf_lucy
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 25])
    .reconstruct(kind='lucy', iterations=20))

stats_lucy = wf_lucy.recon.get_statistics()
print("\nLucy-Richardson:")
print(f"  χ²/dof: {stats_lucy['chi2_per_dof']:.2f}")
print(f"  RMSE: {stats_lucy['rmse']:.2e}")
print(f"  MAE: {stats_lucy['mae']:.2e}")

## Tikhonov Regularization

In [ ]:
wf_tikhonov = Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)

(wf_tikhonov
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 25])
    .reconstruct(kind='tikhonov', noise_power=0.01))

stats_tikhonov = wf_tikhonov.recon.get_statistics()
print("\nTikhonov:")
print(f"  χ²/dof: {stats_tikhonov['chi2_per_dof']:.2f}")
print(f"  RMSE: {stats_tikhonov['rmse']:.2e}")
print(f"  MAE: {stats_tikhonov['mae']:.2e}")

## Visual Comparison

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

methods = [
    ('Wiener', wf_wiener, 'blue'),
    ('Lucy-Richardson', wf_lucy, 'orange'),
    ('Tikhonov', wf_tikhonov, 'green')
]

for ax, (name, wf, color) in zip(axes, methods):
    # Get data
    time_ms = wf.recon.reference_data['time'] / 1000
    reference = wf.recon.reference_data['counts']
    reconstructed = wf.recon.reconstructed_data['counts']
    
    # Plot
    ax.plot(time_ms, reference, 'k-', label='Target', linewidth=2, alpha=0.7)
    ax.plot(time_ms, reconstructed, color=color, label='Reconstructed', 
            linewidth=1.5, linestyle='--')
    
    stats = wf.recon.get_statistics()
    ax.set_title(f"{name} (χ²/dof={stats['chi2_per_dof']:.2f})")
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Counts')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 50)

plt.tight_layout()
plt.show()

## Quantitative Comparison

In [ ]:
import pandas as pd

comparison = pd.DataFrame([
    {'Method': 'Wiener', **stats_wiener},
    {'Method': 'Lucy-Richardson', **stats_lucy},
    {'Method': 'Tikhonov', **stats_tikhonov}
])

print("\nMethod Comparison:")
print(comparison[['Method', 'chi2_per_dof', 'rmse', 'mae', 'max_abs_error']].to_string(index=False))

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

metrics = ['chi2_per_dof', 'rmse', 'mae']
titles = ['χ²/dof (lower is better)', 'RMSE', 'MAE']

for ax, metric, title in zip(axes, metrics, titles):
    comparison.plot(x='Method', y=metric, kind='bar', ax=ax, legend=False)
    ax.set_title(title)
    ax.set_xlabel('')
    ax.set_ylabel(metric)
    ax.grid(True, alpha=0.3, axis='y')
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

## Optimize Each Method

Find optimal parameters for each reconstruction method.

In [ ]:
# Optimize Wiener noise_power
print("Optimizing Wiener filter...")
wiener_sweep = (Workflow('iron_powder.csv', 'openbeam.csv', flux=5e6, duration=0.5, freq=20)
    .convolute(pulse_duration=200)
    .poisson(flux=1e6, freq=60, measurement_time=30)
    .overlap(kernel=[0, 25])
    .groupby('noise_power', low=0.001, high=0.1, num=10)
    .reconstruct(kind='wiener')
    .analyze(xs='iron', vary_background=True, vary_response=True)
    .run())

wiener_clean = wiener_sweep.dropna(subset=['redchi2'])
best_wiener = wiener_clean.loc[wiener_clean['redchi2'].idxmin()]
print(f"  Best noise_power: {best_wiener['noise_power']:.4f}")
print(f"  χ²/dof: {best_wiener['redchi2']:.2f}")

# Note: Lucy-Richardson uses iterations, Tikhonov uses noise_power
# You could similarly sweep these parameters

## Residuals Comparison

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

for ax, (name, wf, color) in zip(axes, methods):
    time_ms = wf.recon.reference_data['time'] / 1000
    reference = wf.recon.reference_data['counts']
    reconstructed = wf.recon.reconstructed_data['counts']
    errors = wf.recon.reference_data['err']
    
    # Calculate residuals in sigma units
    residuals = (reference - reconstructed) / np.maximum(errors, 1e-10)
    
    ax.plot(time_ms, residuals, color=color, alpha=0.7)
    ax.axhline(0, color='black', linestyle='-', linewidth=0.8)
    ax.axhline(2, color='gray', linestyle='--', alpha=0.5)
    ax.axhline(-2, color='gray', linestyle='--', alpha=0.5)
    
    ax.set_title(f"{name} Residuals")
    ax.set_ylabel('Residuals (σ)')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 50)

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.show()

## Key Takeaways

**Wiener Filter**:
- Fast and simple
- Parameter: `noise_power` (balance between denoising and sharpness)
- Good general-purpose choice

**Lucy-Richardson**:
- Iterative method
- Parameter: `iterations` (more = sharper, but can amplify noise)
- Good for positive-valued data

**Tikhonov**:
- Regularization approach
- Parameter: `noise_power` (regularization strength)
- Smoother results, good for noisy data

**Choosing a Method**:
- Compare χ²/dof, RMSE, and residual patterns
- Optimize parameters for each method
- Consider computational cost vs quality trade-off